In [1]:
import tensorflow as tf
from transformers import TFWhisperModel, AutoFeatureExtractor

2024-05-01 01:18:07.702722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 01:18:10.769842: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
SEED = 1337 
ds_train_raw = tf.keras.utils.audio_dataset_from_directory(
    directory="../data/test/",
    batch_size=None,
    seed=SEED,
    output_sequence_length=16000,
)

Found 6835 files belonging to 30 classes.


2024-05-01 01:18:26.758317: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-01 01:18:26.766699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-01 01:18:26.766796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-01 01:18:26.779184: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-01 01:18:26.779272: I external/local_xla/xla/stream_executor

In [3]:
for x, y in ds_train_raw.batch(4).take(1):
    break

In [4]:
class MyModel(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__()
        self.encoder = model
        self.dense1 = tf.keras.layers.Dense(512, activation="relu")
        self.dense2 = tf.keras.layers.Dense(30)

    def call(self, x):
        inputs, decoder_input_ids = x
        x = self.encoder(inputs, decoder_input_ids=decoder_input_ids).last_hidden_state
        x = x[:, -1, :]
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [5]:
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-tiny")
base_model = TFWhisperModel.from_pretrained("openai/whisper-tiny")
model = MyModel(base_model)
model.encoder.trainable = False

All PyTorch model weights were used when initializing TFWhisperModel.

All the weights of TFWhisperModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperModel for predictions without further training.


In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-tiny")
base_model = TFWhisperModel.from_pretrained("openai/whisper-tiny")
model = MyModel(base_model)
model.encoder.trainable = False

ds_train = ds_train_raw.batch(8)

optimizer = tf.keras.optimizers.Lion(learning_rate=1e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


epochs = 10
for epoch in range(epochs):
    epoch_loss = 0
    for x_batch, y_batch in ds_train:  # Assuming train_dataset is properly formatted
        with tf.GradientTape() as tape:
            # Forward pass
            inputs = [feature_extractor(tf.squeeze(x_batch)[i], return_tensors="tf", sampling_rate=16000).input_features for i in range(x_batch.shape[0])]
            inputs = tf.concat(inputs, axis=0)
            print(inputs.shape)

            tmp = [base_model.config.decoder_start_token_id] * x_batch.shape[0] * 2
            decoder_input_ids = tf.convert_to_tensor(tmp)
            decoder_input_ids = tf.reshape(decoder_input_ids, (x_batch.shape[0], 2))

            outputs = model((inputs, decoder_input_ids))

            loss = loss_fn(y_batch, outputs)
            print(loss)
        # Backward pass
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        epoch_loss += loss.numpy()
    
    # Print the average loss for the epoch
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(ds_train)}")


All PyTorch model weights were used when initializing TFWhisperModel.

All the weights of TFWhisperModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperModel for predictions without further training.


(8, 80, 3000)


2024-04-28 19:53:04.338560: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
2024-04-28 19:53:17.057681: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 411.99MiB (rounded to 432000000)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-28 19:53:17.057814: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-04-28 19:53:17.057944: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 40, Chunks in use: 40. 10.0KiB allocated for chunks. 10.0KiB in use in bin. 313B client-requested in use in bin.
2024-04-28 19:53:17.057976: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B 

ResourceExhaustedError: Exception encountered when calling layer 'self_attn' (type TFWhisperAttention).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

Call arguments received by layer 'self_attn' (type TFWhisperAttention):
  • hidden_states=tf.Tensor(shape=(8, 1500, 384), dtype=float32)
  • key_value_states=None
  • past_key_value=None
  • attention_mask=None
  • layer_head_mask=None
  • training=False